- #### Transformaciones
    - ##### Windows

In [ ]:
import org.apache.spark.sql.{SparkSession, DataFrame}

val spark = SparkSession.builder
        .appName("sesion_1")
        .master("local[*]")
        .getOrCreate()
val sc = spark.sparkContext

In [ ]:
sc.uiWebUrl

In [ ]:
def readCsv(path: String): DataFrame = {
    spark.read
        .option("header","true")
        .option("delimiter",",")
        .option("inferSchema","false")
        .csv(path)
}

val BasePath = "../../resources/data/csv/"
val contractsDf = readCsv(BasePath + "contracts.csv")
contractsDf.show()

In [ ]:
// Windows
    // Agregacion -> max, min, count, avg, mean, sum, ...
    // Ranking -> row_number, rank, dense_rank, ...
    // lag lead

import org.apache.spark.sql.expressions.{Window, WindowSpec}
import org.apache.spark.sql.{functions => f}

contractsDf.orderBy(f.col("cod_iuc")).show()

val window_1: WindowSpec = Window.partitionBy(f.col("cod_iuc"))
val window_2: WindowSpec = Window.partitionBy(f.col("cod_iuc")).orderBy(f.col("fec_alta").asc)


contractsDf.select(
    contractsDf.columns.map(f.col) :+
    f.count("*").over(window_1).alias("count_w") :+
    f.max(f.col("fec_alta")).over(window_2).alias("max_fec_alta_w") :+
    f.sum(f.col("cod_producto")).over(window_2).alias("sum_cod_producto_w") :_*
).show()

In [ ]:
contractsDf.orderBy(f.col("cod_iuc")).show()

val window_3: WindowSpec = Window.partitionBy(f.col("cod_iuc")).orderBy(f.col("cod_titular").asc)

contractsDf.select(
    contractsDf.columns.map(f.col) :+
    f.row_number().over(window_3).alias("row_number") :+
    f.rank().over(window_3).alias("rank") :+
    f.dense_rank().over(window_3).alias("dense_rank") :_*
).show(100)

In [ ]:
contractsDf.orderBy(f.col("cod_iuc")).show()

val window_4: WindowSpec = Window.partitionBy(f.col("cod_iuc")).orderBy(f.col("fec_alta").desc)

contractsDf.select(
    contractsDf.columns.map(f.col) :+
    f.lag(f.col("fec_alta"), 1, "1970-01-01").over(window_4).alias("lag") :+
    f.lead(f.col("fec_alta"), 1, "1970-01-01").over(window_4).alias("lead") :_*
).show(100)